# How likely is a Bones Day vs. a No Bones Day?

In [1]:
%load_ext lab_black

### Load Python tools

In [2]:
import pandas as pd
import altair as alt
import datetime as dt

In [3]:
today = dt.datetime.today()

### Read data collected from [TikTok](https://www.tiktok.com/@jongraz) about Noodle

In [ ]:
src = pd.read_csv(
    "https://docs.google.com/spreadsheets/d/e/2PACX-1vQz8eEafXnIeKsNvGL8M56Ia6vu89JDCjfJ22ORl65So4kRYtmTbwmqLjE223fe4mjcZqkrs-KT6j8z/pub?\
gid=0&single=true&output=csv"
)

### Process dates

In [ ]:
src["date"] = pd.to_datetime(src["date"])

In [ ]:
df_date = pd.DataFrame()
index = pd.date_range(start="2021-09-01", end=today, freq="D")
df_date["date"] = index

### Merge the dates to backfill

In [ ]:
df = pd.merge(src, df_date, on="date", how="outer")

In [ ]:
df["weekday"] = df["date"].dt.day_name()
df["month"] = df["date"].dt.month_name()
df["weekend"] = df["date"].dt.day_name().isin(["Saturday", "Sunday"])

In [ ]:
# df.drop(["url"], axis=1, inplace=True)

In [ ]:
df.bones_nobones = df.bones_nobones.fillna("No video")

### What's our dataframe look like? 

In [ ]:
df.head()

### Percentage of days with Bones (only days with a video)

In [ ]:
(
    df[df["bones_nobones"] != "No video"].value_counts("bones_nobones", normalize=True)
    * 100
).round(0)

### Group by day type

In [ ]:
days = (
    df.groupby(["bones_nobones"])
    .agg({"date": "count"})
    .reset_index()
    .rename(columns={"date": "count"})
)

### Simple bar chart explaining the count of day types

In [ ]:
alt.Chart(days[days["bones_nobones"] != "No video"], title=" ").mark_bar().encode(
    y=alt.Y("bones_nobones", title="", axis=alt.Axis(tickCount=0, tickColor="#ffffff")),
    x=alt.X(
        "count",
        title="Days",
        axis=alt.Axis(grid=False, tickCount=6, tickColor="#ffffff"),
    ),
    color=alt.Color(
        "bones_nobones",
        title="",
        legend=None,
        scale=alt.Scale(
            domain=["Bones!", "No bones", "No video"],
            range=["#5ab4ac", "#d8b365", "#e6e6e6"],
        ),
    ),
).properties(width=500, height=70)

### Plot Noodle's days on a calendar

In [ ]:
heatmap = (
    alt.Chart(df[df["date"] > "2021-08-31"], title=" ")
    .mark_rect()
    .encode(
        x=alt.X("date(date):O", title=" "),
        y=alt.Y("month(date):O", title=""),
        color=alt.Color(
            "bones_nobones",
            title="",
            scale=alt.Scale(
                domain=["Bones!", "No bones", "No video"],
                range=["#5ab4ac", "#d8b365", "#e6e6e6"],
            ),
        ),
        tooltip=[
            alt.Tooltip("monthdate(date):T", title="Date"),
            alt.Tooltip("bones_nobones", title="Tweets"),
        ],
    )
    .properties(width=500, height=70, title="Bones days since Sept. 1")
)

heatmap.configure_legend(orient="top", padding=10).configure_view(strokeOpacity=0)

### Prep for datawrapper 

In [ ]:
df.head()

In [ ]:
df["bones_class"] = (
    df["bones_nobones"]
    .str.lower()
    .str.replace(" ", "-", regex=False)
    .str.replace("!", "", regex=False)
)

In [ ]:
df["weekend_class"] = (
    df["weekend"]
    .astype(str)
    .str.lower()
    .str.replace(" ", "-", regex=False)
    .str.replace("!", "", regex=False)
)

In [ ]:
grouped_all = (
    df.groupby("bones_nobones")
    .agg({"date": "size"})
    .reset_index()
    .rename(columns={"date": "Days"})
)

In [ ]:
grouped_all

In [ ]:
grouped_all.iloc[1, 1]

In [ ]:
grouped_weekends = (
    df.groupby(["bones_nobones", "weekend"])
    .agg({"date": "size"})
    .reset_index()
    .rename(columns={"date": "Days"})
)

In [ ]:
grouped_weekends

In [ ]:
grouped_weekends.iloc[3, 2]

In [ ]:
stacked_bar = pd.DataFrame(
    {
        "Category": ["All days", "Weekends", "Weekdays"],
        "Bones": [
            grouped_all.iloc[0, 1],
            grouped_weekends.iloc[1, 2],
            grouped_weekends.iloc[0, 2],
        ],
        "No Bones": [
            grouped_all.iloc[1, 1],
            grouped_weekends.iloc[3, 2],
            grouped_weekends.iloc[2, 2],
        ],
    }
)

In [ ]:
stacked_bar["bones"] = stacked_bar["Bones"]
stacked_bar["nobones"] = stacked_bar["No Bones"]

In [ ]:
stacked_bar

---

In [ ]:
df["week"] = pd.to_datetime(df["date"]).dt.isocalendar().week
df["weekstart"] = pd.to_datetime(df["date"]) - pd.to_timedelta(
    pd.to_datetime(df["date"]).dt.dayofweek, unit="d"
)

In [ ]:
df.head()

In [ ]:
weeks = (
    df.groupby(["weekstart", "bones_class"])
    .agg({"date": "count"})
    .reset_index()
    .rename(columns={"date": "count", "bones_class": "result"})
)

In [ ]:
weeks.head()

In [ ]:
weeks_pivot = pd.pivot_table(
    weeks, values="count", index="weekstart", columns="result", fill_value=0
).reset_index()

In [ ]:
weeks_pivot.to_csv("../_data/bonesdays_weeks.csv", index=False)
weeks_pivot.to_csv("../assets/data/bonesdays_weeks.csv", index=False)

---

In [ ]:
months = (
    df.groupby(["month", "bones_class"])
    .agg({"date": "count"})
    .reset_index()
    .rename(columns={"date": "count", "bones_class": "result"})
)

In [ ]:
months.head()

In [ ]:
months_pivot = pd.pivot_table(
    months, values="count", index="month", columns="result", fill_value=0
).reset_index()

In [ ]:
months_pivot.to_csv("../_data/bonesdays_months.csv", index=False)
months_pivot.to_csv("../assets/data/bonesdays_months.csv", index=False)

---

### Export

In [ ]:
df["display_date"] = df["date"].dt.strftime("%b. %-d ")
df["date"] = df["date"].astype(str)

In [ ]:
stacked_bar.to_csv("../_data/bonesdays_summary_csv.csv", index=False)
stacked_bar.to_csv("../assets/data/bonesdays_summary.csv", index=False)

In [ ]:
df[df["bones_nobones"] != "No video"].to_csv("../_data/bonesdays_csv.csv", index=False)
df[df["bones_nobones"] != "No video"].to_csv(
    "../assets/data/bonesdays.csv", index=False
)

In [ ]:
stacked_bar.to_json("../_data/bonesdays_summary.json", indent=4, orient="records")
stacked_bar.to_json("../assets/data/bonesdays_summary.json", indent=4, orient="records")

In [ ]:
df[df["bones_nobones"] != "No video"].to_json(
    "../_data/bonesdays.json", indent=4, orient="records"
)
df[df["bones_nobones"] != "No video"].to_json(
    "../assets/data/bonesdays.json", indent=4, orient="records"
)

In [ ]:
today = dt.datetime.today().strftime("%Y-%m-%d")
display_today = dt.datetime.today().strftime("%b. %-d")

In [ ]:
toplines = pd.DataFrame(
    {"bones": [grouped_all.iloc[0, 1]], "nobones": [grouped_all.iloc[1, 1]],}
)

In [ ]:
toplines["updated_date"] = today

In [ ]:
toplines

In [ ]:
toplines["display_date"] = display_today

In [ ]:
toplines.to_json("../_data/toplines.json", indent=4, orient="records")
toplines.to_json("../assets/data/toplines.json", indent=4, orient="records")